<a href="https://colab.research.google.com/github/TimH2024/MSC-M5-Project/blob/main/1_Merging_the_Datasets%2C_preliminary_checks_and_adding_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Merging the Datasets, preliminary checks and adding features

---

#### Load Libraries

In [ ]:

!pip install tensorflow
!pip install scikeras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import uniform, randint
import warnings

# Suppress user warnings
warnings.filterwarnings('ignore')

# Adjust settings to display all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the file path
file_path = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/Datasetv2.csv'

# Load the CSV file into a pandas DataFrame called "dataset"
dataset = pd.read_csv(file_path)

# Display the shape of the DataFrame
print("\nDataset Shape:")
print(f"Rows: {dataset.shape[0]}, Columns: {dataset.shape[1]}")

Mounted at /content/drive

Dataset Shape:
Rows: 159676, Columns: 15


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the file path
file_path = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/Property_data_raw_2.csv'

# Load the CSV file into a pandas DataFrame called "proprty_dataset"
property_dataset = pd.read_csv(file_path)

# Display the shape of the DataFrame
print("\nDataset Shape:")
print(f"Rows: {property_dataset.shape[0]}, Columns: {property_dataset.shape[1]}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Dataset Shape:
Rows: 15, Columns: 17


In [ ]:
# Get the files from GitHub raw URLs for the datasets
# dataset_url = 'https://raw.githubusercontent.com/TimH2024/MSC-M5-Project/main/Datasetv2.csv'
# property_dataset_url = 'https://raw.githubusercontent.com/TimH2024/MSC-M5-Project/main/Property_data_raw_2.csv'

# Read in the data sets
# dataset = pd.read_csv(dataset_url)
# property_dataset = pd.read_csv(property_dataset_url)

### Quick Look at the two property datasets

# PROPERTY DATASET (Centerline Properties)

Quick look at the dataset provided by Centaline Properties (essentially a list of 156,676 property transactions in Hong Kong over a period of 3 years)

In [ ]:
dataset.head()

,Unnamed: 0,date,address,price,changes,saleable_area(ft^2),unit_rate,district,Tower,Flat,Phase,Block,Rental,Public Housing,floor
0,0,9/3/2023,"The Long Beach, Tower 7, Middle Floor, Flat D",33000,--,816,40,Kowloon Kowloon City District,7,D,NaN,NaN,True,False,13.0
1,1,9/3/2023,"One Kai Tak, II, Tower 3, Upper Floor, Flat H",15800,--,375,42,Kowloon Kowloon City District,3,H,NaN,NaN,True,False,22.0
2,2,8/3/2023,"Sky Tower, Block 1, Lower Floor, Flat B",18800,--,520,36,Kowloon Kowloon City District,NaN,B,NaN,1,True,False,7.0
3,3,8/3/2023,"Hampton Place, Tower 2, Upper Floor, Flat F",17000,--,429,40,Kowloon Kowloon City District,2,F,NaN,NaN,True,False,22.0
4,4,8/3/2023,"Skypark, Upper Floor, Flat A5",18000,--,407,44,Kowloon Kowloon City District,NaN,A,NaN,NaN,True,False,22.0


In [ ]:
dataset.shape

(159676, 15)

In [ ]:
# Check data types
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159676 entries, 0 to 159675
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           159676 non-null  int64  
 1   date                 159676 non-null  object 
 2   address              159676 non-null  object 
 3   price                159676 non-null  int64  
 4   changes              159676 non-null  object 
 5   saleable_area(ft^2)  159676 non-null  object 
 6   unit_rate            159676 non-null  int64  
 7   district             159676 non-null  object 
 8   Tower                60395 non-null   object 
 9   Flat                 152252 non-null  object 
 10  Phase                48336 non-null   object 
 11  Block                54679 non-null   object 
 12  Rental               159676 non-null  bool   
 13  Public Housing       159676 non-null  bool   
 14  floor                159676 non-null  float64
dtypes: bool(2), float

In [ ]:
# Convert the date column to datetime format and check first and last date in data set
dataset['date'] = pd.to_datetime(dataset['date'], format='%d/%m/%Y')

# Find the first and last dates
first_date = dataset['date'].min()
last_date = dataset['date'].max()

# Display the results
print("First date in the dataset:", first_date)
print("Last date in the dataset:", last_date)

First date in the dataset: 2020-03-10 00:00:00
Last date in the dataset: 2023-03-11 00:00:00


# MARCO INDICIES DATASET

Quick look at the macro economic indicies dataset which contains indicators that could impact the change in property prices. The information is based on statistics provided by the Government of Hong Kong

In [ ]:
property_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   15 non-null     object 
 1   2020Q1       15 non-null     float64
 2   2020Q2       15 non-null     float64
 3   2020Q3       15 non-null     float64
 4   2020Q4       15 non-null     float64
 5   2021Q1       15 non-null     float64
 6   2021Q2       15 non-null     float64
 7   2021Q3       15 non-null     float64
 8   2021Q4       15 non-null     float64
 9   2022Q1       15 non-null     float64
 10  2022Q2       15 non-null     float64
 11  2022Q3       15 non-null     float64
 12  2022Q4       15 non-null     float64
 13  2023Q1       15 non-null     float64
 14  2023Q2       15 non-null     float64
 15  2023Q3       15 non-null     float64
 16  2023Q4       15 non-null     float64
 17  Description  15 non-null     object 
dtypes: float64(16), object(2)
memory usage: 2.2+ KB


In [ ]:
property_dataset.shape

(15, 18)

In [ ]:
# Create a dictionary of descriptions
descriptions = {
    'CPI': 'Consumer Price Index',
    'IR': 'Interest Rate',
    'MW': 'Monthly Wage',
    'M3': 'Money Supply M3',
    'SD': 'Savings Deposit',
    'UR': 'Unemployment Rate',
    'GDP': 'Gross Domestic Product',
    'CI': 'Consumer Confidence Index',
    'SM': 'Stock Market',
    'LTV': 'Loan to Value',
    'HS': 'Housing Starts',
    'SOLD': 'Number of Properties Sold',
    'PG': 'Population Growth',
    'CG': 'China Growth',
    'PI': 'Property price changes'
}

# Map /add descriptions to the dataset_datasetFrame
property_dataset['Description'] = property_dataset['Unnamed: 0'].map(descriptions)

# Display the updated dataset_datasetFrame
property_dataset.head(20)

,Unnamed: 0,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,Description
0,CPI,0.80,0.80,0.80,0.80,0.90,0.90,0.90,0.90,1.80,1.80,1.80,1.80,2.00,2.00,2.00,2.00,Consumer Price Index
1,IR,2.00,0.86,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,2.00,3.50,4.75,5.50,5.75,5.75,Interest Rate
2,MW,17.00,17.00,17.10,16.90,16.80,17.00,17.20,17.20,17.10,17.30,17.50,17.60,17.70,18.00,18.30,18.40,Monthly Wage
3,M3,14.80,14.90,15.90,15.60,15.90,16.00,16.00,16.20,16.50,16.30,16.30,16.40,16.50,16.50,16.80,17.10,Money Supply M3
4,SD,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.08,0.08,0.08,0.08,0.08,Savings Deposit
5,UR,3.70,5.90,6.20,6.20,7.20,6.00,4.60,4.10,4.50,5.10,4.10,3.70,3.30,3.00,2.90,2.90,Unemployment Rate
6,GDP,86.24,86.24,86.24,86.24,92.23,92.23,92.23,92.23,89.96,89.96,89.96,89.96,91.40,91.40,91.40,91.40,Gross Domestic Product
7,CI,62.90,52.30,59.80,63.80,66.60,66.80,68.00,63.40,50.60,53.70,43.90,28.30,66.10,61.00,52.60,51.00,Consumer Confidence Index
8,SM,27.90,24.20,25.10,26.50,28.90,29.10,24.80,23.99,21.91,20.72,17.75,19.90,20.58,19.45,17.95,17.45,Stock Market
9,LTV,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,Loan to Value


# MERGE THE DATASETS

### Merge both datasets and switch around columns of the macro economic indicies dataset

In [ ]:
from datetime import datetime

# Convert the date column to datetime format
dataset['date'] = pd.to_datetime(dataset['date'], format='%d/%m/%Y')

# Define the function to convert date to year-quarter format
def get_year_quarter(date):
    year = date.year
    quarter = (date.month - 1) // 3 + 1
    return date.strftime('%d-%m-%Y'), f"{year}Q{quarter}"

# Extract formatted date and quarter info from the dataset
dataset[['FormattedDate', 'YearQuarter']] = dataset['date'].apply(lambda x: pd.Series(get_year_quarter(x)))

# Melt the property dataset to have a long format
property_dataset_melted = property_dataset.melt(id_vars=[property_dataset.columns[0]], var_name='YearQuarter', value_name='Value')

# Pivot the melted property dataset to have a wide format
property_dataset_pivoted = property_dataset_melted.pivot(index='YearQuarter', columns=property_dataset.columns[0], values='Value').reset_index()

# Merge the datasets on the YearQuarter column
merged_dataset = pd.merge(dataset, property_dataset_pivoted, on='YearQuarter', how='left')

# Display the merged dataset
print(merged_dataset.head())

   Unnamed: 0       date                                        address  \
0           0 2023-03-09  The Long Beach, Tower 7, Middle Floor, Flat D   
1           1 2023-03-09  One Kai Tak, II, Tower 3, Upper Floor, Flat H   
2           2 2023-03-08        Sky Tower, Block 1, Lower Floor, Flat B   
3           3 2023-03-08    Hampton Place, Tower 2, Upper Floor, Flat F   
4           4 2023-03-08                  Skypark, Upper Floor, Flat A5   

   price changes saleable_area(ft^2)  unit_rate  \
0  33000      --                 816         40   
1  15800      --                 375         42   
2  18800      --                 520         36   
3  17000      --                 429         40   
4  18000      --                 407         44   

                        district Tower Flat Phase Block  Rental  \
0  Kowloon Kowloon City District     7    D   NaN   NaN    True   
1  Kowloon Kowloon City District     3    H   NaN   NaN    True   
2  Kowloon Kowloon City District   NaN   

## Preliminary review of the merged dataset

In [ ]:
merged_dataset.shape

(159676, 32)

In [ ]:
# Look at the data types in the merged_database
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159676 entries, 0 to 159675
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Unnamed: 0           159676 non-null  int64         
 1   date                 159676 non-null  datetime64[ns]
 2   address              159676 non-null  object        
 3   price                159676 non-null  int64         
 4   changes              159676 non-null  object        
 5   saleable_area(ft^2)  159676 non-null  object        
 6   unit_rate            159676 non-null  int64         
 7   district             159676 non-null  object        
 8   Tower                60395 non-null   object        
 9   Flat                 152252 non-null  object        
 10  Phase                48336 non-null   object        
 11  Block                54679 non-null   object        
 12  Rental               159676 non-null  bool          
 13  Public Housing

In [ ]:
# Check year quarter has worked
merged_dataset['YearQuarter'].unique()

array(['2023Q1', '2022Q4', '2022Q3', '2022Q2', '2022Q1', '2021Q4',
       '2021Q3', '2021Q2', '2021Q1', '2020Q4', '2020Q3', '2020Q2',
       '2020Q1'], dtype=object)

In [ ]:
# Drop unnamed column
merged_dataset.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
# Save the merged dataset to a new CSV file (unprocessed merged dataset)
merged_dataset.to_csv('Merged_Datasetv1.csv', index=False)

### Quick initial checks

In [ ]:
null_counts = merged_dataset.isnull().sum()

# Display the number of null values for each column
print("Number of null values for each column:")
print(null_counts)

Number of null values for each column:
date                        0
address                     0
price                       0
changes                     0
saleable_area(ft^2)         0
unit_rate                   0
district                    0
Tower                   99281
Flat                     7424
Phase                  111340
Block                  104997
Rental                      0
Public Housing              0
floor                       0
FormattedDate               0
YearQuarter                 0
CG                          0
CI                          0
CPI                         0
GDP                         0
HS                          0
IR                          0
LTV                         0
M3                          0
MW                          0
PG                          0
PI                          0
SD                          0
SM                          0
SOLD                        0
UR                          0
dtype: int64


There are significant numbers of null values in some of the address fields on some of the properties which is expects as some properties will not have towere, flat, phase and block in their addreses. These will have to be deleted later

In [ ]:
# Count the number of duplicate rows
duplicates_count = merged_dataset.duplicated().sum()
print(f"Number of duplicate rows: {duplicates_count}")

# Remove duplicate rows
merged_dataset.drop_duplicates(inplace=True)

# Shape of the DataFrame after removing duplicates
merged_dataset.shape

Number of duplicate rows: 64


(159612, 31)

## Removing  'Rental' and 'Public Housing'


### How many properties sold were are Public Housing" as opposed to private housing

In [ ]:
# Count the number of rows where 'Public Housing' is True
true_count = merged_dataset['Public Housing'].sum()

print("Number of True rows in 'Public Housing':", true_count)

Number of True rows in 'Public Housing': 1553


In [ ]:
# Check the shape of the dataset before deleting rows where 'Rental' is True
print("Shape before deletion:", merged_dataset.shape)

# Delete rows where 'Rental' is True
merged_dataset = merged_dataset[merged_dataset['Public Housing'] != True]

# Check the shape of the dataset after deletion
print("Shape after deletion:", merged_dataset.shape)

# Count the number of rows where 'Public Housing' is True (i.e., properties sold that are Public Housing)
public_housing_count = merged_dataset['Public Housing'].sum()

print("Number of properties sold that are Public Housing:", public_housing_count)

Shape before deletion: (159612, 31)
Shape after deletion: (158059, 31)
Number of properties sold that are Public Housing: 0


## How any rows in the dataset are Rental properties = True (this means they are rental properties)


In [ ]:
# Count the number of rows where 'Rental' is True
true_count = merged_dataset['Rental'].sum()

print("Number of True rows in 'Rental':", true_count)
# We need to delete these as we are focusing on property purchase

Number of True rows in 'Rental': 48208


In [ ]:
# Count the number of rows where 'Rental' is True (i.e., rental properties)
rental_count = merged_dataset['Rental'].sum()

print("Number of True rows in 'Rental' (Rental properties):", rental_count)

# Check the shape of the dataset before deleting rows where 'Rental' is True
print("Shape before deletion:", merged_dataset.shape)

# Delete rows where 'Rental' is True
merged_dataset = merged_dataset[merged_dataset['Rental'] != True]

# Check the shape of the dataset after deletion
print("Shape after deletion:", merged_dataset.shape)

# Count the number of properties that are not rentals (i.e., purchased properties)
purchased_count = len(merged_dataset)

print("Number of properties sold (non-rental):", purchased_count)

Number of True rows in 'Rental' (Rental properties): 48208
Shape before deletion: (158059, 31)
Shape after deletion: (109851, 31)
Number of properties sold (non-rental): 109851


In [ ]:
# Check the shape of the dataset before deleting columns 'Rental' and 'Public Housing'
print("Shape before column deletion:", merged_dataset.shape)

# Delete the columns 'Rental' and 'Public Housing'
merged_dataset = merged_dataset.drop(columns=['Rental', 'Public Housing'])

# Check the shape of the dataset after deleting the columns
print("Shape after column deletion:", merged_dataset.shape)

Shape before column deletion: (109851, 31)
Shape after column deletion: (109851, 29)


In [ ]:
# Check data types after conversion and make sure macro features are float
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109851 entries, 7 to 159675
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date                 109851 non-null  datetime64[ns]
 1   address              109851 non-null  object        
 2   price                109851 non-null  int64         
 3   changes              109851 non-null  object        
 4   saleable_area(ft^2)  109851 non-null  object        
 5   unit_rate            109851 non-null  int64         
 6   district             109851 non-null  object        
 7   Tower                37274 non-null   object        
 8   Flat                 103459 non-null  object        
 9   Phase                31329 non-null   object        
 10  Block                38474 non-null   object        
 11  floor                109851 non-null  float64       
 12  FormattedDate        109851 non-null  object        
 13  YearQuarter        

In [ ]:
# Drop unnecessary and null value columns
columns_to_drop = ['Tower', 'Flat', 'Phase', 'Block', 'address', 'changes','date']
merged_dataset = merged_dataset.drop(columns=columns_to_drop)

# Verify the columns have been dropped by checking the shape
print("Shape of the dataset after dropping columns:", merged_dataset.shape)

# Optionally, display the first few rows to confirm the columns are removed
print("new shape after deletions", merged_dataset.shape)

Shape of the dataset after dropping columns: (109851, 22)
new shape after deletions (109851, 22)


In [ ]:
# Convert macro database indicies which are objects to float for future ML

# Define the columns to be converted to floats
columns_to_convert = ['CG', 'CI', 'CPI', 'GDP', 'HS', 'IR', 'LTV', 'M3', 'MW', 'PG', 'PI', 'SD', 'SM', 'SOLD', 'UR']

# Convert the specified columns to floats
for column in columns_to_convert:
    merged_dataset[column] = pd.to_numeric(merged_dataset[column], errors='coerce').astype(float)

# Display the first few rows to ensure conversion
#print(merged_dataset.head())
merged_dataset.head()

,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
13,76000000,"2,190",34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3


In [ ]:
# Now double check the number of null values for each column after conversion
null_counts = merged_dataset.isnull().sum()

# Display the number of null values for each column
print("Number of null values for each column:")
print(null_counts)

Number of null values for each column:
price                  0
saleable_area(ft^2)    0
unit_rate              0
district               0
floor                  0
FormattedDate          0
YearQuarter            0
CG                     0
CI                     0
CPI                    0
GDP                    0
HS                     0
IR                     0
LTV                    0
M3                     0
MW                     0
PG                     0
PI                     0
SD                     0
SM                     0
SOLD                   0
UR                     0
dtype: int64


In [ ]:
merged_dataset.shape

(109851, 22)

In [ ]:
#Check columns nbames
merged_dataset.columns

Index(['price', 'saleable_area(ft^2)', 'unit_rate', 'district', 'floor',
       'FormattedDate', 'YearQuarter', 'CG', 'CI', 'CPI', 'GDP', 'HS', 'IR',
       'LTV', 'M3', 'MW', 'PG', 'PI', 'SD', 'SM', 'SOLD', 'UR'],
      dtype='object')

In [ ]:
merged_dataset.head()

,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
13,76000000,"2,190",34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3


# FEATURE ENGINEERING

In [ ]:
# Look at the types of districts in the database
merged_dataset['district'].unique()

array(['Kowloon Kowloon City District', 'Kowloon Kwun Tong District',
       'Kowloon Yau Tsim Mong District', 'Kowloon Wong Tai Sin District',
       'Kowloon Sham Shui Po District',
       'New Territories East Sha Tin District',
       'New Territories East Long Ping Estate',
       'New Territories East North District',
       'New Territories East Tai Po District', 'Kwai Tsing District',
       'Tsuen Wan District', 'Yuen Long District', 'Tuen Mun District',
       'New Territories West Islands District', 'Wan Chai District',
       'HKIsIand Southern District', 'Central and Western District',
       'HKIsIand Eastern District'], dtype=object)

In [ ]:
# Define the district_region mapping
district_region = {
    'Kowloon Kowloon City District': 'KLN',
    'Kowloon Kwun Tong District': 'KLN',
    'Kowloon Yau Tsim Mong District': 'KLN',
    'Kowloon Wong Tai Sin District': 'KLN',
    'New Territories East Long Ping Estate' 'NTEast'
    'Kowloon Sham Shui Po District': 'KLN',
    'New Territories East Sha Tin District': 'NTEast',
    'New Territories East North District': 'NTEast',
    'New Territories East Tai Po District': 'NTEast',
    'Kwai Tsing District': 'NTWest',
    'Tsuen Wan District': 'NTWest',
    'Tuen Mun District': 'NTWest',
    'New Territories West Islands District': 'NTWest',
    'Wan Chai District': 'HK',
    'HKIsland Southern District': 'HK',
    'Central and Western District': 'HK',
    'HKIsland Eastern District': 'HK'
}

# Map the 'district' column to create the 'region' column
merged_dataset['region'] = merged_dataset['district'].map(district_region)

# Check for unmapped districts (optional)
unmapped_districts = merged_dataset[merged_dataset['region'].isna()]['district'].unique()
if len(unmapped_districts) > 0:
    print("Unmapped districts:", unmapped_districts)

# Display the first few rows of the updated dataset
display(merged_dataset.head())

Unmapped districts: ['Kowloon Sham Shui Po District' 'New Territories East Long Ping Estate'
 'Yuen Long District' 'HKIsIand Southern District'
 'HKIsIand Eastern District']


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN
13,76000000,"2,190",34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN


In [ ]:
# Apply one-hot encoding
encoded_districts = pd.get_dummies(merged_dataset['district'], prefix='district')

# Merge the one-hot encoded columns with the original dataset
merged_dataset = pd.concat([merged_dataset, encoded_districts], axis=1)

# Explicitly set one-hot encoded columns to int64
merged_dataset[encoded_districts.columns] = merged_dataset[encoded_districts.columns].astype('int64')

# Retain the original 'district' column
print("Displaying the first few rows with original column retained:")
display(merged_dataset.head())

Displaying the first few rows with original column retained:


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,76000000,"2,190",34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109851 entries, 7 to 159675
Data columns (total 41 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   price                                           109851 non-null  int64  
 1   saleable_area(ft^2)                             109851 non-null  object 
 2   unit_rate                                       109851 non-null  int64  
 3   district                                        109851 non-null  object 
 4   floor                                           109851 non-null  float64
 5   FormattedDate                                   109851 non-null  object 
 6   YearQuarter                                     109851 non-null  object 
 7   CG                                              109851 non-null  float64
 8   CI                                              109851 non-null  float64
 9   CPI                            

In [ ]:
# Apply one-hot encoding for the 'region' column
encoded_region = pd.get_dummies(merged_dataset['region'], prefix='region')

# Merge the one-hot encoded columns with the original dataset
merged_dataset = pd.concat([merged_dataset, encoded_region], axis=1)

# Explicitly set one-hot encoded columns to int64
merged_dataset[encoded_region.columns] = encoded_region.astype('int64')

# Do NOT drop the original 'region' column (keep it in the dataset)

# Display the first few rows of the updated dataset
print("Displaying the first 5 rows of the updated dataset with original 'region' column retained:")
display(merged_dataset.head())

Displaying the first 5 rows of the updated dataset with original 'region' column retained:


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District,region_HK,region_KLN,region_NTEast,region_NTWest
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
13,76000000,"2,190",34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Create property sizes small, medium, large and very large
# Function to remove comma from column and convert price strings to numeric
def convert_numeric(value):
    if isinstance(value, str):
        # Remove commas and convert to numeric, handling errors by setting to NaN
        return pd.to_numeric(value.replace(',', ''), errors='coerce')
    return value

# Apply the conversion to the 'saleable_area(ft^2)' column
merged_dataset['saleable_area(ft^2)'] = merged_dataset['saleable_area(ft^2)'].apply(convert_numeric)

# Categorize properties by size and add category column
def categorize_size(area):
    if area < 500:
        return 'Small'
    elif 500 <= area <= 1000:
        return 'Medium'
    elif 1000 <= area <= 1500:
        return 'Large'
    else:
        return 'Very Large'

# Apply the categorize_size function to the 'saleable_area(ft^2)' column and assign the result to the 'property_size' column
merged_dataset['property_size'] = merged_dataset['saleable_area(ft^2)'].apply(categorize_size)

In [ ]:
# Apply one-hot encoding for the 'property_size' column
encoded_property_size = pd.get_dummies(merged_dataset['property_size'], prefix='property_size')

# Merge the one-hot encoded columns with the original dataset
merged_dataset = pd.concat([merged_dataset, encoded_property_size], axis=1)

# Explicitly set one-hot encoded columns to int64
merged_dataset[encoded_property_size.columns] = encoded_property_size.astype('int64')

# Do NOT drop the original 'property_size' column (keep it in the dataset)

# Display the first few rows of the updated dataset
print("Displaying the first 5 rows of the updated dataset with original 'property_size' column retained:")
display(merged_dataset.head())

Displaying the first 5 rows of the updated dataset with original 'property_size' column retained:


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District,region_HK,region_KLN,region_NTEast,region_NTWest,property_size,property_size_Large,property_size_Medium,property_size_Small,property_size_Very Large
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0
13,76000000,2190,34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Very Large,0,0,0,1
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0


In [ ]:
# Apply one-hot encoding for the 'YearQuarter' column
encoded_YearQuarter = pd.get_dummies(merged_dataset['YearQuarter'], prefix='YearQuarter')

# Merge the one-hot encoded columns with the original dataset
merged_dataset = pd.concat([merged_dataset, encoded_YearQuarter], axis=1)

# Explicitly set one-hot encoded columns to int64
merged_dataset[encoded_YearQuarter.columns] = encoded_YearQuarter.astype('int64')

# Do NOT drop the original 'YearQuarter' column (keep it in the dataset)

# Display the first few rows of the updated dataset
print("Displaying the first 5 rows of the updated dataset with original 'YearQuarter' column retained:")
display(merged_dataset.head())

Displaying the first 5 rows of the updated dataset with original 'YearQuarter' column retained:


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District,region_HK,region_KLN,region_NTEast,region_NTWest,property_size,property_size_Large,property_size_Medium,property_size_Small,property_size_Very Large,YearQuarter_2020Q1,YearQuarter_2020Q2,YearQuarter_2020Q3,YearQuarter_2020Q4,YearQuarter_2021Q1,YearQuarter_2021Q2,YearQuarter_2021Q3,YearQuarter_2021Q4,YearQuarter_2022Q1,YearQuarter_2022Q2,YearQuarter_2022Q3,YearQuarter_2022Q4,YearQuarter_2023Q1
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13,76000000,2190,34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Very Large,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Fixing the Price (which was a rounded number)

In [ ]:
# Clean the dataset
merged_dataset['saleable_area(ft^2)'] = merged_dataset['saleable_area(ft^2)'].astype(str).str.replace(',', '')  # Remove commas
merged_dataset['saleable_area(ft^2)'] = merged_dataset['saleable_area(ft^2)'].str.extract('(\d+\.?\d*)')[0]  # Extract numeric part

# Convert to float
merged_dataset['saleable_area(ft^2)'] = pd.to_numeric(merged_dataset['saleable_area(ft^2)'], errors='coerce')

# Display cleaned dataset types
print("Cleaned Dataset Types")
print(merged_dataset.info())

# Check for remaining null values
print("Number of Null Values After Conversion:", merged_dataset['saleable_area(ft^2)'].isnull().sum())

Cleaned Dataset Types
<class 'pandas.core.frame.DataFrame'>
Index: 109851 entries, 7 to 159675
Data columns (total 63 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   price                                           109851 non-null  int64  
 1   saleable_area(ft^2)                             109851 non-null  int64  
 2   unit_rate                                       109851 non-null  int64  
 3   district                                        109851 non-null  object 
 4   floor                                           109851 non-null  float64
 5   FormattedDate                                   109851 non-null  object 
 6   YearQuarter                                     109851 non-null  object 
 7   CG                                              109851 non-null  float64
 8   CI                                              109851 non-null  float64
 9   CPI      

In [ ]:
# Now create a new_price column with corrected figures
# Add a new 'price' column calculated as 'saleable_area(ft^2)' * 'unit_rate'
merged_dataset['new_price'] = merged_dataset['saleable_area(ft^2)'] * merged_dataset['unit_rate']

# Print the first 100 rows of the dataset_datasetFrame
print("First 20 rows of the dataset_datasetFrame with the new 'price' column:")
display(merged_dataset.head(20))

First 20 rows of the dataset_datasetFrame with the new 'price' column:


,price,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District,region_HK,region_KLN,region_NTEast,region_NTWest,property_size,property_size_Large,property_size_Medium,property_size_Small,property_size_Very Large,YearQuarter_2020Q1,YearQuarter_2020Q2,YearQuarter_2020Q3,YearQuarter_2020Q4,YearQuarter_2021Q1,YearQuarter_2021Q2,YearQuarter_2021Q3,YearQuarter_2021Q4,YearQuarter_2022Q1,YearQuarter_2022Q2,YearQuarter_2022Q3,YearQuarter_2022Q4,YearQuarter_2023Q1,new_price
7,6680000,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6680016
13,76000000,2190,34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Very Large,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,75999570
14,12070000,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12067177
15,4400000,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4389440
16,8600000,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8584779
17,10300000,699,14721,Kowloon Kowloon City District,24.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10289979
18,7380000,463,15907,Kowloon Kowloon City District,19.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7364941
19,11150000,593,18799,Kowloon Kowloon City District,30.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,11147807
20,7580000,501,15111,Kowloon Kowloon City District,11.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7570611
21,6000000,439,13662,Kowloon Kowloon City District,23.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5997618


### Now drop the old price column from the datset

In [ ]:
# Display the dataset_datasetFrame before dropping the column
#print("Before Dropping 'price' Column:")
#print(merged_dataset)

# Drop the 'price' column
merged_dataset = merged_dataset.drop(columns=['price'])


merged_dataset.head(5)

,saleable_area(ft^2),unit_rate,district,floor,FormattedDate,YearQuarter,CG,CI,CPI,GDP,HS,IR,LTV,M3,MW,PG,PI,SD,SM,SOLD,UR,region,district_Central and Western District,district_HKIsIand Eastern District,district_HKIsIand Southern District,district_Kowloon Kowloon City District,district_Kowloon Kwun Tong District,district_Kowloon Sham Shui Po District,district_Kowloon Wong Tai Sin District,district_Kowloon Yau Tsim Mong District,district_Kwai Tsing District,district_New Territories East Long Ping Estate,district_New Territories East North District,district_New Territories East Sha Tin District,district_New Territories East Tai Po District,district_New Territories West Islands District,district_Tsuen Wan District,district_Tuen Mun District,district_Wan Chai District,district_Yuen Long District,region_HK,region_KLN,region_NTEast,region_NTWest,property_size,property_size_Large,property_size_Medium,property_size_Small,property_size_Very Large,YearQuarter_2020Q1,YearQuarter_2020Q2,YearQuarter_2020Q3,YearQuarter_2020Q4,YearQuarter_2021Q1,YearQuarter_2021Q2,YearQuarter_2021Q3,YearQuarter_2021Q4,YearQuarter_2022Q1,YearQuarter_2022Q2,YearQuarter_2022Q3,YearQuarter_2022Q4,YearQuarter_2023Q1,new_price
7,423,15792,Kowloon Kowloon City District,22.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6680016
13,2190,34703,Kowloon Kowloon City District,7.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Very Large,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,75999570
14,761,15857,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12067177
15,320,13717,Kowloon Kowloon City District,12.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Small,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4389440
16,519,16541,Kowloon Kowloon City District,29.0,08-03-2023,2023Q1,9.1,66.1,2.0,91.4,5250.0,4.75,0.5,16.5,17.7,8.0,-10.24,0.08,20.58,14509.0,3.3,KLN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Medium,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8584779


In [ ]:
# Doble check on the null values
#merged_dataset.info()
merged_dataset.isnull().sum()

,0
saleable_area(ft^2),0
unit_rate,0
district,0
floor,0
FormattedDate,0
YearQuarter,0
CG,0
CI,0
CPI,0
GDP,0


In [ ]:
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109851 entries, 7 to 159675
Data columns (total 63 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   saleable_area(ft^2)                             109851 non-null  int64  
 1   unit_rate                                       109851 non-null  int64  
 2   district                                        109851 non-null  object 
 3   floor                                           109851 non-null  float64
 4   FormattedDate                                   109851 non-null  object 
 5   YearQuarter                                     109851 non-null  object 
 6   CG                                              109851 non-null  float64
 7   CI                                              109851 non-null  float64
 8   CPI                                             109851 non-null  float64
 9   GDP                            

In [ ]:
merged_dataset.shape

(109851, 63)

In [ ]:
# Save the merged dataset to a new CSV file
merged_dataset.to_csv('Merged_Datasetv2.csv', index=False)

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Define the file path
file_path = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/Merged_Datasetv2.csv'

# Save the file
merged_dataset.to_csv(file_path, index=False)

# Verify if the file exists
if os.path.exists(file_path):
    print(f"File 'Merged_Datasetv2.csv' has been saved successfully!")
    print(f"Full file path: {file_path}")
else:
    print(f"File 'Merged_Datasetv2.csv' was not found in the directory!")

# Optionally unmount and remount Google Drive
drive.flush_and_unmount()
print("Drive unmounted. Please refresh Google Drive and remount if necessary.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File 'Merged_Datasetv2.csv' has been saved successfully!
Full file path: /content/drive/My Drive/Colab Notebooks/M5 Code and Data/Merged_Datasetv2.csv
Drive unmounted. Please refresh Google Drive and remount if necessary.
